In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

/Users/chi/.venvs/env_torch/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

def preprocess_function(examples):
    return tokenizer(examples['cleaned_review_text'], padding='max_length', truncation=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset('json', data_files={'train': '/Users/chi/Documents/imdb_spoiler/sampled_preprocessed.json'})
# Split 80% Train, 20% Temp (Validation + Test)
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Further split the 20% Temp into 10% Validation and 10% Test
val_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

final_dataset = {
    "train": train_test_split["train"],
    "val": val_test_split["train"],  # 10% validation
    "test": val_test_split["test"],  # 10% test
}

tokenized_datasets = {split: data.map(preprocess_function, batched=True) for split, data in final_dataset.items()}


Map: 100%|██████████| 283/283 [00:00<00:00, 1095.42 examples/s]


In [5]:
# Training arguments
training_args = TrainingArguments(
    output_dir='saves/results_gpt2',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='saves/logs_gpt2',
)

In [15]:
tokenized_datasets.keys()

dict_keys(['train'])

In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
)

/var/folders/4x/qtt50c_16_j8ntstjbyx20v40000gn/T/ipykernel_9780/1216322619.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
trainer.train()
results_gpt2 = trainer.evaluate()
print(f"GPT-2 Evaluation Loss: {results_gpt2['eval_loss']:.4f}")

RuntimeError: MPS backend out of memory (MPS allocated: 13.71 GB, other allocations: 4.52 GB, max allowed: 18.13 GB). Tried to allocate 768.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).